In [1]:
import pandas as pd
from langdetect import detect
pd.set_option('display.max_columns', None)

In [2]:
filepath = './data/BeerAdvocate/'

In [3]:
beers = pd.read_csv(filepath + 'beers.csv')
beers.head(3)

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,nbr_reviews,avg,ba_score,bros_score,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
0,166064,Nashe Moskovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.7,NaN,NaN,0,NaN
1,166065,Nashe Pivovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,3.8,NaN,NaN,0,NaN
2,166066,Nashe Shakhterskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.8,NaN,NaN,0,NaN


In [4]:
reviews = pd.read_csv(filepath + 'reviews.csv')
reviews.head(3)

,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating,text
0,Régab,142544,Societe des Brasseries du Gabon (SOBRAGA),37262,Euro Pale Lager,4.5,1440064800,nmann08,nmann08.184925,3.25,2.75,3.25,2.75,3.0,2.88,"From a bottle, pours a piss yellow color with ..."
1,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,1235127600,StJamesGate,stjamesgate.163714,3.00,3.50,3.50,4.00,3.5,3.67,Pours pale copper with a thin head that quickl...
2,Barelegs Brew,19590,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.5,1142247600,mdagnew,mdagnew.19527,4.00,3.50,3.50,4.00,3.5,3.73,"500ml Bottle bought from The Vintage, Antrim....."


In [5]:
breweries = pd.read_csv(filepath + 'breweries.csv')
breweries.head(3)

,id,location,name,nbr_beers
0,39912,Kyrgyzstan,Abdysh-Ata (Абдыш Ата),5
1,39913,Kyrgyzstan,Arpa (АРПА),1
2,39914,Kyrgyzstan,Bear Beer,4


In [6]:
users = pd.read_csv(filepath + 'users.csv')
users.head(3)

,nbr_ratings,nbr_reviews,user_id,user_name,joined,location
0,7820,465,nmann08.184925,nmann08,1.199704e+09,"United States, Washington"
1,2521,2504,stjamesgate.163714,StJamesGate,1.191838e+09,"United States, New York"
2,1797,1143,mdagnew.19527,mdagnew,1.116410e+09,Northern Ireland


In [7]:
# rename columns of users to have user_ prefix
users.rename(columns={'nbr_ratings':'user_nbr_ratings', 
                      'nbr_reviews':'user_nbr_reviews', 
                      'joined':'user_joined', 
                      'location':'user_location'}, inplace=True)

# Merge dfs
reviews_users = pd.merge(reviews, users, how='left', on=['user_id', 'user_name'])

# Drop NaN values
reviews_users.dropna(inplace=True)

print('We have {} valid reviews.'.format(reviews_users.shape[0]))

We have 2400932 valid reviews.


In [8]:
#rename the columns of beers to contain beer_ prefix
beers.rename(columns={'nbr_ratings':'beer_nbr_ratings', 
                      'nbr_reviews':'beer_nbr_reviews', 
                      'ba_score':'beer_ba_score', 
                      'bros_score':'beer_bros_score', 
                      'avg':'beer_avg',
                      'avg_computed':'beer_avg_computed',
                      'zscore':'beer_zscore',}, 
             inplace=True)

# merge reviews with some columns of beers
reviews_users_beers = pd.merge(reviews_users, beers[['beer_id', 'beer_name','beer_nbr_ratings', 'beer_nbr_reviews', 'beer_ba_score', 'beer_avg']], 
                               how='left', 
                               on=['beer_id', 'beer_name'])

# drop NaN from reviews_users_beers
reviews_users_beers.dropna(inplace=True)

print('We have {} valid reviews'.format(reviews_users_beers.shape[0]))

We have 2258373 valid reviews


In [9]:
# rename columns of breweries to contain brewery_ prefix
breweries.rename(columns={'id':'brewery_id', 
                          'location':'brewery_location',
                          'name':'brewery_name',
                          'nbr_beers':'brewery_nbr_beers',
                          }, 
                 inplace=True)

# merge reviews with breweries
reviews_users_beers_breweries = pd.merge(reviews_users_beers, breweries, 
                               how='left', 
                               on=['brewery_id', 'brewery_name'])

# drop NaN from reviews_users_beers_breweries
reviews_users_beers_breweries.dropna(inplace=True)

print('We have {} valid reviews'.format(reviews_users_beers_breweries.shape[0]))

We have 2174716 valid reviews


In [11]:
reviews_users_beers_breweries.sample(10)

,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating,text,user_nbr_ratings,user_nbr_reviews,user_joined,user_location,beer_nbr_ratings,beer_nbr_reviews,beer_ba_score,beer_avg,brewery_location,brewery_nbr_beers
285371,Dogbolter,3891,Matilda Bay Brewing Co.,1480,Munich Dunkel Lager,5.2,1022580000,rastaman,rastaman.264,4.00,3.50,4.00,3.5,3.50,3.58,"A very handy little dark lager, nice roasty co...",1355,1208,1.013857e+09,England,34.0,26.0,84.0,3.68,Australia,29.0
941063,077-07871 (Mosaic),155319,Carton Brewing Company,26817,American Double / Imperial IPA,7.8,1423825200,Dustin_Sallitt,dustin_sallitt.917340,4.75,5.00,4.50,4.5,4.75,4.69,Was lucky enough to pick up a growler full fro...,349,212,1.420024e+09,"United States, New Jersey",81.0,20.0,93.0,4.40,"United States, New Jersey",87.0
189248,Beer Geek Vanilla Shake,98050,Mikkeller ApS,13307,American Double / Imperial Stout,13.0,1480503600,brureview,brureview.655761,4.00,4.25,4.25,4.5,4.25,4.34,No brew date. BB on the cap 10/02/26.Poured a ...,778,761,1.327057e+09,"United States, Massachusetts",978.0,168.0,95.0,4.32,Denmark,795.0
2100426,Cantillon Bruocsella (1900) Grand Cru,2559,Brasserie Cantillon,388,Lambic - Unblended,5.0,1247738400,Nickls,nickls.162218,4.00,4.50,4.00,4.0,4.00,4.12,"(From notes)Thanks, Joe!A - Hazy orange-yellow...",468,468,1.191319e+09,"United States, Washington",857.0,320.0,87.0,3.90,Belgium,82.0
937667,Abbey Dubbel,1027,Flying Fish Brewing Company,339,Dubbel,7.2,1044010800,cbl2,cbl2.1445,3.00,4.00,3.50,4.0,4.00,3.89,Appearance: Crystal brownSmell: Sweet &amp; ma...,661,661,1.043838e+09,"United States, New York",734.0,300.0,81.0,3.55,"United States, New Jersey",69.0
1668980,The Dogfather,45777,Laughing Dog Brewing,12985,American Double / Imperial Stout,11.0,1457434800,BeerBobber,beerbobber.1071668,4.25,4.00,4.25,4.0,4.00,4.04,This beer was poured from a 22 oz. bottle into...,243,243,1.448276e+09,"United States, Nevada",580.0,248.0,86.0,3.85,"United States, Idaho",38.0
939482,Grand Cru Winter Reserve,1960,Flying Fish Brewing Company,339,Belgian Strong Pale Ale,7.2,1261911600,TheFightfan1,thefightfan1.273469,3.50,4.50,4.00,4.5,4.00,4.29,Poured into a snifter. Poured a slightly hazy ...,1339,643,1.228302e+09,"United States, Pennsylvania",571.0,233.0,83.0,3.68,"United States, New Jersey",69.0
97960,Aphrodisiaque,10330,Brasserie Dieu du Ciel!,1141,American Stout,6.5,1264935600,bret27,bret27.306820,4.50,4.00,4.50,4.5,4.50,4.38,"bottle courtesy of myself.S: Roasty, chocolate...",169,169,1.236683e+09,"United States, California",1662.0,633.0,93.0,4.19,Canada,242.0
2109319,La Chouffe,1836,Brasserie d'Achouffe,321,Belgian Strong Pale Ale,8.0,1300964400,wvsabbath,wvsabbath.454130,4.50,4.00,3.50,4.0,4.00,3.98,Serving Type - BottleAppearance - Light orange...,1545,873,1.272535e+09,"United States, West Virginia",2562.0,905.0,93.0,4.16,Belgium,20.0
658069,8 Ball Stout,5045,Lost Coast Brewery and Cafe,256,Oatmeal Stout,6.3,1056880800,Morris729,morris729.996,4.00,4.00,4.50,4.5,4.50,4.35,Pours pitch black in color with a creamy brown...,701,701,1.036840e+09,"United States, Texas",1083.0,552.0,86.0,3.85,"United States, California",63.0


In [24]:
# Save to csv
# reviews_users_beers_breweries.to_csv(filepath + 'BeerAdvocateMerged.csv', index=False)

### Language detection

In [ ]:
# detect language of reviews in BeerAdvocate
## must sample because langdetect takes too long to run on all reviews

# reviews_ba_sample = reviews_ba.sample(n=5000, random_state=46)
# reviews_ba_sample['detected_language'] = reviews_ba_sample['text'].apply(lambda x: detect(x) if isinstance(x, str) else '')

# # show the percentage of reviews in each language
# reviews_ba_sample['detected_language'].value_counts(normalize=True)